In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
labels.shape

(6920,)

In [5]:
texts = df[0].values.tolist()

In [7]:
from transformers import AutoTokenizer, TFGPT2ForSequenceClassification

In [35]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")
model = TFGPT2ForSequenceClassification.from_pretrained("microsoft/DialogRPT-updown", from_pt=True)

All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

All the weights of TFGPT2ForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2ForSequenceClassification for predictions without further training.


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=0)

In [37]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [38]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [39]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_gpt2"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [40]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 3035s 76s/step - loss: 0.5878 - accuracy: 0.6319 - val_loss: 0.3677 - val_accuracy: 0.8375
Epoch 2/10
39/39 [==============================] - 3326s 85s/step - loss: 0.3244 - accuracy: 0.8577 - val_loss: 0.2832 - val_accuracy: 0.8755
Epoch 3/10
39/39 [==============================] - 3589s 93s/step - loss: 0.2204 - accuracy: 0.9121 - val_loss: 0.2737 - val_accuracy: 0.8863
Epoch 4/10
39/39 [==============================] - 3332s 85s/step - loss: 0.1494 - accuracy: 0.9448 - val_loss: 0.2578 - val_accuracy: 0.8899
Epoch 5/10
39/39 [==============================] - 4281s 110s/step - loss: 0.1070 - accuracy: 0.9613 - val_loss: 0.2822 - val_accuracy: 0.8935
Epoch 6/10
39/39 [==============================] - 3762s 97s/step - loss: 0.0695 - accuracy: 0.9749 - val_loss: 0.3186 - val_accuracy: 0.9007
Epoch 7/10
39/39 [==============================] - 4145s 106s/step - loss: 0.0509 - accuracy: 0.9811 - val_loss: 0.3715 - val_accuracy: 0.89

In [ ]:
y_preds = model.predict(dict(X_test_tokenized))

44/44 [==============================] - 170s 4s/step


In [26]:
y_preds.logits

array([[-3.5683296],
       [-6.1167245],
       [ 3.4054523],
       ...,
       [ 6.943949 ],
       [ 6.7827125],
       [ 3.8216572]], dtype=float32)

In [29]:
prediction_probs=tf.nn.sigmoid(y_preds.logits).numpy()

In [31]:
y_predictions = [1 if x > 0.5 else 0 for x in prediction_probs ]

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88       635
           1       0.90      0.89      0.89       749

    accuracy                           0.89      1384
   macro avg       0.88      0.88      0.88      1384
weighted avg       0.89      0.89      0.89      1384

